<a href="https://colab.research.google.com/github/VinodKumar9576/ML-Problems/blob/master/Stock Screener/Bullish RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests-html

     |████████████████████████████████| 153kB 7.1MB/s 
     |████████████████████████████████| 143kB 21.7MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=09ed64ee6ceb4995503a4f64f7d4122992680df7a619f91021acd6d1b795353c
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24134 sha256=3460a8ffc04b4dff91560c88b9c977e23f9a5420e16573c40979377bbabe157e
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built fake-useragent parse
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have fo

In [ ]:
!pip install telegram-send
#!telegram-send --configure

     |████████████████████████████████| 430kB 6.6MB/s 
     |████████████████████████████████| 61kB 4.2MB/s 
     |████████████████████████████████| 2.6MB 24.4MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import telegram_send
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import six

#Code for telegram messages
telegram_token = '1361504293:AAEdPRnT4HNMmPSzUHjB9ClWLdrH9dfZWOE'
chat_id = '-1001495643344'
path_config = telegram_send.get_config_path()
with open(path_config, 'w') as f:
  f.write(f'[telegram]\ntoken = {telegram_token}\nchat_id = {chat_id}')

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


* Code to extract MIS stocks in Zerodha

In [ ]:
src_url = 'https://zerodha.com/margin-calculator/Equity/'
with requests.Session() as s:
    r = s.get(src_url)
    soup = BeautifulSoup(r.text, "html.parser")

div_cont = soup.find('div', {'id': 'header-container'})
table_div = div_cont.find('table', {'id':'table'})
table_rows = table_div.find_all('tr')

mis_ls = []
for tr in table_rows:
  try:
    td = tr.find('td')
    row = td.text
    mis_ls.append(row)
  except:
    None

mis_stocks = pd.DataFrame(data = mis_ls, columns = ['Scrip'])

* Conditions for scanner

In [ ]:
#For positional trade (rsi include DTF, WKTF, MOTF candles)
pos_multi_RSI = '(+{cash}+(+latest+close+>=+300+and+latest+volume+<=+500000+and+latest+adx(+14+)+>=+30+and(+{cash}+(+latest+rsi(+14+)+>+70+and+1+day+ago++rsi(+14+)+<=+70+or+weekly+rsi(+14+)+>+70+and+1+week+ago++rsi(+14+)+<=+70+or+monthly+rsi(+14+)+>+70+and+1+month+ago++rsi(+14+)+<=+70+)+)+and+latest+close+>=+latest+sma(+close,50+)+and+latest+macd+histogram(+26,12,9+)+>=+1+day+ago+macd+histogram(+26,12,9+)+and+latest+close+<=+4000+and+latest+volume+>+20+)+)+'
pos_multi_RSI = pos_multi_RSI.replace('+', ' ')



#(rsi include only Daily candle)
intra_RSI_D = '(+{cash}+(+latest+close+>=+300+and+latest+volume+<=+500000+and+latest+adx(+14+)+>=+30+and(+{cash}+(+latest+rsi(+14+)+>+70+and+1+day+ago++rsi(+14+)+<=+70+)+)+and+latest+close+>=+latest+sma(+close,50+)+and+latest+macd+histogram(+26,12,9+)+>=+1+day+ago+macd+histogram(+26,12,9+)+and+latest+close+<=+4000+and+latest+volume+>+20+)+)+'
intra_RSI_D = intra_RSI_D.replace('+', ' ')


#For Intraday considering just 5min candle along with other conditions
intra_RSI = '(+{cash}+(+latest+close+>=+300+and+latest+volume+<=+500000+and+latest+adx(+14+)+>=+30+and(+{cash}+(+[0]+5+minute+rsi(+14+)+>+70+and+[+-1+]+5+minute+rsi(+14+)+<=+70+)+)+and+latest+close+>=+latest+sma(+close,50+)+and+latest+macd+histogram(+26,12,9+)+>=+1+day+ago+macd+histogram(+26,12,9+)+and+latest+close+<=+4000+and+latest+volume+>+20+)+)+'
intra_RSI = intra_RSI.replace('+', ' ')



#Include 5min, dtf, wktf, motf candles
#With below we get stocks that cross rsi > 70 and the stocks currently also
#trading with RSI>70 & at 10:25 we get day RSI broke stocks
comp_intra = '(+{cash}+(+latest+close+>=+300+and+latest+volume+<=+500000+and+latest+adx(+14+)+>=+30+and(+{cash}+(+latest+rsi(+14+)+>+70+and+1+day+ago++rsi(+14+)+<=+70+or+weekly+rsi(+14+)+>+70+and+1+week+ago++rsi(+14+)+<=+70+or+monthly+rsi(+14+)+>+70+and+1+month+ago++rsi(+14+)+<=+70+or+[0]+5+minute+rsi(+14+)+>+70+and+[+-1+]+5+minute+rsi(+14+)+<=+70+)+)+and+latest+close+>=+latest+sma(+close,50+)+and+latest+macd+histogram(+26,12,9+)+>=+1+day+ago+macd+histogram(+26,12,9+)+and+latest+close+<=+4000+and+latest+volume+>+20+)+)+'
comp_intra = comp_intra.replace('+', ' ')


#Just rsi>70 in 5min candle
just_rsi = '(+{cash}+(+latest+close+>=+300+and+latest+volume+<=+500000+and(+{cash}+(+[0]+5+minute+rsi(+14+)+>+66.6+and+[+-1+]+5+minute+rsi(+14+)+<=+66.6+)+)+and+latest+close+<=+4000+and+latest+volume+>+20+)+)+'
just_rsi = just_rsi.replace('+', ' ')


#cross-over scanner
#22DEMA cross above 50SMA & 30HMA cross above 50SMA
#fad_cross_over = '(+{cash}+(+[0]+5+minute+ema(+[0]+5+minute+close+,+22+)+>+[0]+5+minute+sma(+close,50+)+and+[+-1+]+5+minute+ema(+[0]+5+minute+close+,+22+)<=+[+-1+]+5+minute+sma(+close,50+)+and+[0]+5+minute+"wma(+(+(+2+*+wma(+([0]+5+minute+close),+15)+)+-+wma(([0]+5+minute+close),+30)+),+5)"+>+[0]+5+minute+sma(+[0]+5+minute+close+,+50+)+and+[+-1+]+5+minute+"wma(+(+(+2+*+wma(+([0]+5+minute+close),+15)+)+-+wma(([0]+5+minute+close),+30)+),+5)"+<=+[+-1+]+5+minute+sma(+[0]+5+minute+close+,+50+)+)+)+'
#fad_cross_over = fad_cross_over.replace('+', ' ')

fad_cross_over = '(+{cash}+(+[0]+5+minute+ema(+[0]+5+minute+close+,+22+)+>+[0]+5+minute+sma(+close,50+)+and+[+-1+]+5+minute+ema(+[0]+5+minute+close+,+22+)<=+[+-1+]+5+minute+sma(+close,50+)+and+[0]+5+minute+"wma(+(+(+2+*+wma(+([0]+5+minute+close),+15)+)+-+wma(([0]+5+minute+close),+30)+),+5)"+>+[0]+5+minute+sma(+close,50+)+and+[+-1+]+5+minute+"wma(+(+(+2+*+wma(+([0]+5+minute+close),+15)+)+-+wma(([0]+5+minute+close),+30)+),+5)"+<=+[+-1+]+5+minute+sma(+close,50+)+and+latest+close+>+150+)+)+'
fad_cross_over = fad_cross_over.replace('+', ' ')


#15min candle cross over
# 9EMA crosses 22SMA
fift_cross_over = '(+{cash}+(+[0]+15+minute+sma(+close,9+)+>+[0]+15+minute+sma(+close,22+)+and+[+-1+]+15+minute+sma(+close,9+)<=+[+-1+]+15+minute+sma(+close,22+)+and+latest+close+>+200+)+)+'

Tomorrow try 'comp_intra' if don't get much opportunities, try 'intra_RSI'

In [ ]:
#has to run only once in an entire day
sym_set_prev = set()

#Parameters & Selection criteria
src_url = "https://chartink.com/screener/rsi-14-wd-cross-above-60"
base_url = 'https://chartink.com/screener/process'

payload = {
    'scan_clause':	fad_cross_over
}

* Should maintain atleast 2 min gap between each requests, as we are not getting response due to 'Too many requests'.

In [ ]:
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    plt.close(fig)
    return fig

In [ ]:
def temp_func(sym_set_prev):
  def BULLISH_RSI(payload, src_url, base_url):
    stk_sym_ls = []
    stk_sym_now = set()

    with requests.Session() as s:
      r = s.get(src_url)
      soup = BeautifulSoup(r.text, "html.parser")
      csrf = soup.select_one("[name='csrf-token']")['content']
      s.headers['x-csrf-token'] = csrf
      r = s.post(base_url, data = payload)

      for item in r.json()['data']:
        stk_sym_now.add(item['nsecode'])
        stk_sym_ls.append([item['nsecode'],item['close'],item['volume']])
    
      #stk_sym_now = stk_sym_now - stk_sym_prev
      #stk_sym_prev = stk_sym_now
      stk_sym_df = pd.DataFrame(stk_sym_ls, columns = ['Symbol', 'Close', 'Volume'])
  
    return stk_sym_now, stk_sym_df


  #To get the stocks & to filter out from previous run
  sym_set_now, sym_df = BULLISH_RSI(payload, src_url, base_url)
  sym_set_now = sym_set_now - sym_set_prev
  sym_set_prev = sym_set_now

  #To send messages only when stocks are filtered.
  if(bool(sym_set_now)):
    sym_df = sym_df[sym_df['Symbol'].isin(sym_set_now)].sort_values(['Symbol'])
    
    #making mis allowed flag column
    sym_df['Mis_allowed'] = sym_df['Symbol'].isin(mis_stocks['Scrip']).astype(int)
    sym_df.replace({'Mis_allowed':{1:'Yes', 0:'No'}}, inplace = True)

    #making serial column
    sym_df['Sl.No'] = np.arange(1, len(sym_df)+1)
    
    #moving serial column to first position
    cols = sym_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    sym_df = sym_df[cols]


    #sending stocks as image
    fig = render_mpl_table(sym_df, header_columns=0, col_width=2.0)
    fig.savefig('Bullish_RSI_Stocks.png', bbox_inches = 'tight', pad_inches = 0)

    #sending image to telegram
    with open("Bullish_RSI_Stocks.png", "rb") as f:
      telegram_send.send(images=[f], captions = ['Bullish Intraday Stocks that crossed RSI 70'])

    #taking extra space so removing below and sending stocks as text
    #Formatting the df values
    #sym_df = sym_df.applymap(str)
    #sym_df['Symbol'] = 'Symbol : ' + sym_df['Symbol']
    #sym_df['Close'] = 'Price : ' + sym_df['Close']
    #sym_df['Volume'] = 'Volume : ' + sym_df['Volume']

    #sym_str = sym_df.agg('\n'.join, axis = 1)
    #sym_dtls = ['\n\n'.join(sym_str)]
    
    #tm = [time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime(time.time()+5.5*60*60))]
    #tm.extend(sym_dtls)
    #finl_st = ['\n\n'.join(tm)]
    #telegram_send.send(messages = finl_st)
  
  return sym_set_prev

In [ ]:
import sched, time
s = sched.scheduler(time.time, time.sleep)

def run_screener(s, sym_set_prev, t):
  sym_prev_out = temp_func(sym_set_prev)
  sym_set_prev = sym_prev_out

  t = t+(10/60)*60*60
  s.enterabs(t, 1, run_screener, (s,sym_set_prev,t))

#Don't forget to change the date every day, else code will run for every second if date is less than today date
t = time.strptime('2020-12-29 10:20:00', '%Y-%m-%d %H:%M:%S')
t = time.mktime(t)-5.5*60*60

s.enterabs(t, 1, run_screener, (s,sym_set_prev,t))
s.run()

KeyboardInterrupt: ignored

In [ ]:
sym_prev_out = temp_func(sym_set_prev)
sym_set_prev = sym_prev_out

In [ ]:
import sched
import time as time_module

def myfunc(scheduler,t):
  print("Working")
  t = t+(1/12)*60*60
  scheduler.enterabs(t, 1, myfunc, (scheduler, t))

scheduler = sched.scheduler(time_module.time, time_module.sleep)

t = time_module.strptime('2020-12-04 09:15:00', '%Y-%m-%d %H:%M:%S')
t = time_module.mktime(t)-5.5*60*60
scheduler_e = scheduler.enterabs(t, 1, myfunc, (scheduler, t))
scheduler.run()